In [1]:
from knmi.downloader import KNMIDownloader
from knmi.database import KNMIDatabase
from knmi.processor import KNMIProcessor
from config import xlBasisfile, dbFile 

# Instantieer classes
db = KNMIDatabase(dbFile, xlBasisfile)
    
downloader = KNMIDownloader(db)
processor = KNMIProcessor(db)

# als de opgegeven database-file niet bestaat, wordt een nieuwe database aangemaakt
if not db.exists():
    db.create_database()
    db.fill_database()

# de database wordt (aan)gevuld met de meest recente data
#downloader.update_data()

#db.get_last_observations


In [2]:
import pandas as pd
import sqlite3

qry_station_data = '''SELECT date(Timestamp) AS Date, Value, Quality 
                             FROM KNMI_data 
                             WHERE StationId = ? AND Parameter = ? AND Timestamp BETWEEN ? AND ?;
                       '''


StatID = '6'
Param = 'P'
startdate = '2020-01-01'
enddate = '2020-01-10'
LocationID = 'De Kooy'

#start = pd.to_datetime(startdate).to_julian_date()
#end = pd.to_datetime(enddate).to_julian_date()

#with sqlite3.connect(db.db_path) as conn:
#    qry_result = pd.read_sql_query(qry_station_data, conn, params = (StatID, Param, start, end))

#def get_location_timeseries(self, LocationID, start_date, end_date):

stats = db.get_location_stations(LocationID)

stats

,StationId,Parameter,valid_from,valid_through,first_date,last_date,obs_from,obs_through
0,2,P,1-1-1973,None,1958-04-01,2025-11-30,1973-01-01,2025-11-30
1,3,P,1-1-1906,31-12-1972,1851-01-01,1972-12-31,1906-01-01,1972-12-31
2,16,E,1-1-1901,31-12-2000,1957-07-01,2025-12-21,1957-07-01,2000-12-31
3,17,E,1-1-2001,None,1964-11-01,2025-12-21,2001-01-01,2025-12-21


In [3]:
processor.Filter.set_filter(locations= ['De Kooy', 'De Bilt'], startyear= 1990, endyear=2020,startmonth=3,endmonth=9)

print(processor.Filter)

KNMI data filter object (locations: ['De Kooy', 'De Bilt'], years: 1990 - 2020, months: 3 - 9)


In [4]:
db.get_location_timeseries(LocationID,'P','2020-01-01', '2020-12-31')

c:\Users\brinkm03\OneDrive - Rijkswaterstaat\Documenten\01 Projecten\data-analyse\knmi-p13\knmi\database.py:289: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result,self.get_station_timeseries(stat['StationId'],Param,


,Value,Quality
date,,
2020-01-01,0.0,None
2020-01-02,0.3,None
2020-01-03,6.5,None
2020-01-04,6.5,None
2020-01-05,1.2,None
...,...,...
2020-12-27,4.0,None
2020-12-28,10.2,None
2020-12-29,0.3,None


In [4]:
processor.set_filter(locations= 'all', startyear= 1990, endyear=2020,startmonth=4,endmonth=9)

dfdp = processor.get_filtered_PDseries()

#dfdp['avg'] = dfdp.mean(axis=1)

#df_mean = dfdp.mean(axis=1)

dfdp

c:\Users\brinkm03\OneDrive - Rijkswaterstaat\Documenten\01 Projecten\data-analyse\knmi-p13\knmi\database.py:289: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result,self.get_station_timeseries(stat['StationId'],Param,


date
1990-04-01    -2.700000
1990-04-02     0.000000
1990-04-03     1.746154
1990-04-04     3.361538
1990-04-05     2.492308
                ...    
2020-09-26    21.907692
2020-09-27    38.415385
2020-09-28    41.084615
2020-09-29    44.715385
2020-09-30    45.453846
Length: 4940, dtype: float64

In [23]:
def cumsum_with_min(s, minimum=0):
    sl = list(s)
    
    res = []
    for i in range(len(sl)):
        if i == 0:
            res.append(sl[i])
        else:
            res.append(max(minimum,sl[i] +  res[-1]))

    if isinstance(s, pd.Series):
        return pd.Series(res, index=s.index)
    else:
        return res

cumsum_with_min([1,1,2,-3,-2,1,-4,1,3,1,4],2)

[1, 2, 4, 2, 2, 3, 2, 3, 6, 7, 11]

In [17]:
res =dfdp['avg'].groupby(dfdp.index.year).apply(cumsum_with_min,0).droplevel(0)[200:250]

res

date
1990-09-17    28.746154
1990-09-18    33.169231
1990-09-19    33.546154
1990-09-20    40.423077
1990-09-21    49.961538
1990-09-22    51.369231
1990-09-23    53.784615
1990-09-24    66.953846
1990-09-25    72.138462
1990-09-26    72.146154
1990-09-27    71.930769
1990-09-28    70.769231
1990-09-29    69.738462
1990-09-30    75.292308
1991-03-01     1.807692
1991-03-02     5.669231
1991-03-03     4.961538
1991-03-04     3.661538
1991-03-05     6.169231
1991-03-06     6.830769
1991-03-07     5.946154
1991-03-08     5.461538
1991-03-09     5.492308
1991-03-10     4.330769
1991-03-11     3.807692
1991-03-12     2.430769
1991-03-13     0.530769
1991-03-14     0.000000
1991-03-15     0.007692
1991-03-16     0.000000
1991-03-17     0.000000
1991-03-18     0.000000
1991-03-19     4.200000
1991-03-20     4.053846
1991-03-21     6.592308
1991-03-22     6.230769
1991-03-23     4.453846
1991-03-24     2.469231
1991-03-25     0.269231
1991-03-26     0.000000
1991-03-27     0.000000
1991-03-28 

In [24]:
df_mean.groupby(df_mean.index.year).apply(cumsum_with_min,0).droplevel(0)

date
1990-03-01    18.715385
1990-03-02    23.169231
1990-03-03    24.107692
1990-03-04    23.730769
1990-03-05    23.146154
                ...    
2020-09-26    21.907692
2020-09-27    38.415385
2020-09-28    41.084615
2020-09-29    44.715385
2020-09-30    45.453846
Length: 5777, dtype: float64

In [7]:
t2= processor.db.get_station_timeseries('1','P','2020-01-01','2020-02-10')

t2

,Value,Quality
date,,
2020-01-01,0.1,None
2020-01-02,0.1,None
2020-01-03,0.6,None
2020-01-04,4.2,None
2020-01-05,1.1,None
2020-01-06,0.1,None
2020-01-07,0.4,None
2020-01-08,0.5,None
2020-01-09,5.3,None
